In [97]:
import pandas as pd

In [98]:
df=pd.read_csv("G:/ml/sms-spam-collection-dataset/spam.csv", encoding='latin-1')

In [99]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [100]:
# df=df[2:]

In [101]:
df=df[["v1","v2"]]

In [102]:
df.shape

(5572, 2)

In [103]:
# y=df[["v1"]]
# X=df[["v2"]]

In [104]:
import string
from nltk.corpus import stopwords

In [105]:
def text_process(mess):
    nopunc=[char for char in mess if char not in string.punctuation]
    text=''.join(nopunc)
    return [word for word in text.split() if word.lower() not in stopwords.words('english')]

In [106]:
# df["without_stopwords"]=df.apply(lambda x: text_process(x["v2"]),axis=1)

In [107]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [108]:
from sklearn.feature_extraction.text import CountVectorizer

In [109]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(df['v2'])

In [110]:
print(len(bow_transformer.vocabulary_))

11304


In [111]:
messages_bow = bow_transformer.transform(df["v2"])

In [112]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(messages_bow)

In [113]:
messages_tfidf=tfidf_transformer.transform(messages_bow)

In [114]:
messages_tfidf.shape

(5572, 11304)

In [115]:
from sklearn.naive_bayes import MultinomialNB

In [116]:
spam_detect_model = MultinomialNB().fit(messages_tfidf,df['v1'])

In [117]:
all_predictions=spam_detect_model.predict(messages_tfidf) 

In [118]:
print(all_predictions)

['ham' 'ham' 'spam' ..., 'ham' 'ham' 'ham']


In [119]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(df['v1'],all_predictions))

             precision    recall  f1-score   support

        ham       0.98      1.00      0.99      4825
       spam       1.00      0.85      0.92       747

avg / total       0.98      0.98      0.98      5572



In [120]:
from sklearn.model_selection import train_test_split

In [121]:
msg_train,msg_test,label_train,label_test = train_test_split(df['v2'],df['v1'],test_size=0.3)

In [122]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
   ( 'bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB()),
])

In [123]:
pipeline.fit(msg_train,label_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x0000007620BAA840>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocesso...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [124]:
predictions = pipeline.predict(msg_test)

In [125]:
print(classification_report(predictions,label_test))

             precision    recall  f1-score   support

        ham       1.00      0.95      0.98      1523
       spam       0.68      1.00      0.81       149

avg / total       0.97      0.96      0.96      1672

